In [1]:
import openai
openai.api_key = "sk-8fuZdYFOxSDQDfi3nbwhGH3fOJzZFDiezVwvIPBc"

In [2]:
from app import apiWrapper
print("Enter Input Question")
qry = input()
qry += " "
while qry.isspace():
    qry = input("Enter Input Question")

print("Would you like to print intermediate code results? yes/no")
selection = input()
while selection not in ["yes", "no"]:
    selection = input()
    print("Would you like to print intermediate code results? yes/no")

apiWrapper.getUsrQues(qry)
RAW_TEXT, Query = apiWrapper.callApi()
Query = Query.replace("|","")
print("Query Received is ", Query)
print("Solution Generated")

Enter Input Question
integrate from -4 to 4: sqrt(16-x^2)
Would you like to print intermediate code results? yes/no
yes
Query Received is  integral_(-4)^4 sqrt(16 - x^2) dx = 8 π≈25.133
Solution Generated


In [3]:
if selection == "yes":
    for line in RAW_TEXT:
        print(line)

Compute the definite integral:
 integral_(-4)^4 sqrt(16 - x^2) dx
For the integrand sqrt(16 - x^2), substitute x = 4 sin(u) and dx = 4 cos(u) du.
Then sqrt(16 - x^2) = sqrt(16 - 16 sin^2(u)) = 4 sqrt(cos^2(u)).
This substitution is invertible over -π/2<u<π/2 with inverse u = sin^(-1)(x/4).
This gives a new lower bound u = sin^(-1)(-4/4) = -π/2 and upper bound u = sin^(-1)(4/4) = π/2:
 = 4 integral_(-π/2)^(π/2) 4 cos(u) sqrt(cos^2(u)) du
Factor out constants:
 = 16 integral_(-π/2)^(π/2) cos(u) sqrt(cos^2(u)) du
Simplify cos(u) sqrt(cos^2(u)) assuming -π/2<u<π/2:
 = 16 integral_(-π/2)^(π/2) cos^2(u) du
Write cos^2(u) as 1/2 cos(2 u) + 1/2:
 = 16 integral_(-π/2)^(π/2) (1/2 cos(2 u) + 1/2) du
Integrate the sum term by term and factor out constants:
 = 8 integral_(-π/2)^(π/2) cos(2 u) du + 8 integral_(-π/2)^(π/2) 1 du
For the integrand cos(2 u), substitute s = 2 u and ds = 2 du.
This gives a new lower bound s = (2 (-π))/2 = -π and upper bound s = (2 π)/2 = π:
 = 4 integral_(-π)^π cos(s) ds 

In [4]:
import os
import sys
sys.path.append(os.getcwd())

from api import GPT, Example, UIConfig
from api import demo_web_app
from glob import glob
def read_file(path_to_file):
    retval = ""
    file = open(path_to_file)
    retval = file.readlines()
    file.close()
    #Make sure the new line character is not read it throws the model off     
    retval = [x.split("/n")[0][:-1] for x in retval]
    return retval


# Construct GPT object and show some examples
gpt = GPT(engine="davinci",
          temperature=0.01,
          max_tokens=300)


# reade file and convert it to source string and target string tuples
source_names = [item for item in sorted(glob("./Training_Example/text2latex/sources/*"))]
target_names = [item for item in sorted(glob("./Training_Example/text2latex/latex/*"))]


# open each file in the Training_Example directory
for src_path, target_path in zip(source_names,target_names):
    
    # For each files read the RAW and corrosponding Latex Code
    src_RAW = read_file(src_path)
    target_RAW = read_file(target_path)
    
    # for each pair of RAW and latex prime the GPT model
    if len(src_RAW) != len(target_RAW):
        raise Exception("Source and Latex have mismached number of line {} {} in file {} and {}".format(str(len(src_RAW)), str(len(target_RAW)),src_path,target_path))

    for s_RAW, t_RAW in zip(src_RAW,target_RAW):
        gpt.add_example(Example(s_RAW,t_RAW))
        # Uncomment the following if you would like to see the priming examples
        #print("Source: ", s_RAW)
        #print("Output: ", t_RAW)
        #print("----")
        
print("")

In [5]:
import os
import sys
sys.path.append(os.getcwd())

from api import GPT, Example, UIConfig
from api import demo_web_app
from glob import glob
def read_file(path_to_file):
    retval = ""
    file = open(path_to_file)
    retval = file.readlines()
    file.close()
    #Make sure the new line character is not read it throws the model off     
    retval = [x.split("/n")[0][:-1] for x in retval]
    return retval


# Construct GPT object and show some examples
gpt_py = GPT(engine="davinci",
          temperature=0.01,
          max_tokens=100)


# reade file and convert it to source string and target string tuples
source_names = [item for item in sorted(glob("./Training_Example/text2py/sources/*"))]
target_names = [item for item in sorted(glob("./Training_Example/text2py/python/*"))]


# open each file in the Training_Example directory
for src_path, target_path in zip(source_names,target_names):
    
    # For each files read the RAW and corrosponding Latex Code
    src_RAW = read_file(src_path)
    target_RAW = read_file(target_path)
    
    # for each pair of RAW and latex prime the GPT model
    if len(src_RAW) != len(target_RAW):
        raise Exception("Source and Latex have mismached number of line {} {} in file {} and {}".format(str(len(src_RAW)), str(len(target_RAW)),src_path,target_path))

    for s_RAW, t_RAW in zip(src_RAW,target_RAW):
        gpt_py.add_example(Example(s_RAW,t_RAW))
        # Uncomment the following if you would like to see the priming examples
        #print("Source: ", s_RAW)
        #print("Output: ", t_RAW)
        #print("----")
        
print("")

In [6]:
# Converting RAW_TEXT Query to Python Function:
print("Attempting to convert input query to graphable python function")
python_func = gpt_py.get_top_reply(Query)
python_func = python_func[7:]
print("Interpereted python function is", python_func)

Attempting to convert input query to graphable python function
Interpereted python function is  -4*math.sqrt(16 - x**2)**4*math.pi




In [7]:
# Converting RAW_TEXT to Latex:
from tqdm.auto import tqdm
print("Fetching the intermediate LateX code from OpenAI GPT3 API")
response = []
for i in tqdm(range(len(RAW_TEXT))) :
    line = RAW_TEXT[i]
    t = gpt.get_top_reply(line)
    response.append(t)
print("Intermediate LateX generated")

Fetching the intermediate LateX code from OpenAI GPT3 API



Intermediate LateX generated


In [8]:
latex_code = []
for line in response:
    text = line.split("\n")[0][7:]
    if text.isspace() or text == "":
        continue
    else:
        latex_code.append(text + "\n")

In [9]:
if selection == "yes":
    for line in latex_code:
        print(line, end="")

 Compute \ the \ definite \ integral:
 \int_{-4}^{4} \sqrt{16 - x^{2}} dx
 For \ the \ integrand \ sqrt{16 - x^{2}}, \ substitute \ x = 4 \sin(u) \ and \ dx = 4 \cos(u) \ du.
 Then \sqrt{16 - x^{2}} = \sqrt{16 - 16 \sin^2(u)} = 4 \sqrt{ \cos^2(u)}.
 This \ substitution \ is \ invertible \ over \ -\frac{π}{2}<u<\frac{π}{2} \ with \ inverse \ u = \sin^{-1}(x/4).
 This \ gives \ a \ new \ lower \ bound \ u = \sin^{-1}(-4/4) = -\frac{\pi}{2} \ and \ upper \ bound \ u = \sin^{-1}(4/4) = \frac{\pi}{2}:
 = 4 \int_{-\frac{\pi}{2}}^{\frac{\pi}{2}} 4 \cos(u) \sqrt{\cos^2(u)} du
 Factor \ out \ constants:
 = 16 \int_{-\frac{\pi}{2}}^{\frac{\pi}{2}} \cos(u) \sqrt{\cos^2(u)} du
 Simplify \ cos(u) \sqrt{ \cos^{2}(u)} \ assuming \ -\frac{\pi}{2}<u<\frac{\pi}{2}
 = 16 \int_{-\frac{\pi}{2}}^{\frac{\pi}{2}} \cos^2(u) du
 Write \cos^{2}(u) \ as \ 1/2 \cos(2 u) + 1/2:
 = 16 \int_{-\frac{\pi}{2}}^{\frac{\pi}{2}} (\frac{1}{2} \cos(2 u) + \frac{1}{2}) du
 Integrate \ the \ sum \ term \ by \ term \ and \ fact

In [32]:
from app import latex2Manim
import importlib
importlib.reload(latex2Manim)
print("Converting Latex to Maxnim Code")
manim_code = latex2Manim.latex2Manim(latex_code,Query ,python_func)
if selection == "yes":
    print(manim_code)
print("Manim Code Generated")

Converting Latex to Maxnim Code
Do you want to generate a graph for integral_(-4)^4 sqrt(16 - x^2) dx = 8 π≈25.133? yes/no
no
from manimlib.imports import *
from math import *
class Solution(GraphScene):
	def construct(self):
		watermark = ImageMobject("./assets/water_mark.png")
		watermark.scale(1.5)
		watermark.to_corner(DOWN+RIGHT, buff=0)
		self.play(FadeIn(watermark))
		Solve = TexMobject(r" Compute \ the \ definite \ integral:")
		Solve.to_edge(UP)
		self.play(Write(Solve))
		self.wait(2)
		R0 = TexMobject(r" \int_{-4}^{4} \sqrt{16 - x^{2}} dx")
		R1 = TexMobject(r" For \ the \ integrand \ sqrt{16 - x^{2}}, \ substitute \ x = 4 \sin(u) \ and \ dx = 4 \cos(u) \ du.")
		R2 = TexMobject(r" Then \sqrt{16 - x^{2}} = \sqrt{16 - 16 \sin^2(u)} = 4 \sqrt{ \cos^2(u)}.")
		R3 = TexMobject(r" This \ substitution \ is \ invertible \ over \ -\frac{π}{2}<u<\frac{π}{2} \ with \ inverse \ u = \sin^{-1}(x/4).")
		R4 = TexMobject(r" This \ gives \ a \ new \ lower \ bound \ u = \sin^{-1}(-4/4) = -\f

In [33]:
fptr =  open("solution.py", "w") 
fptr.write(manim_code)
fptr.close()
print("Manim Code saved at ./solution.py")

Manim Code saved at ./solution.py


In [35]:
!manim solution.py Solution -pl --media_dir "./Animations"

Media will be written to ./Animations/. You can change this behavior with the --media_dir flag.
Writing "This \ substitution \ is \ invertible \ over \ - \frac{π}{2}<u<\frac{π}{2} \ with \ inverse \ u = \sin^{-1}(x/4)." to ./Animations/Tex/36a9c53e265450c6.tex



Traceback (most recent call last):
  File "/home/mrselukar/miniconda3/lib/python3.8/site-packages/manimlib/extract_scene.py", line 155, in main
    scene = SceneClass(**scene_kwargs)
  File "/home/mrselukar/miniconda3/lib/python3.8/site-packages/manimlib/scene/scene.py", line 75, in __init__
    self.construct()
  File "solution.py", line 16, in construct
    R3 = TexMobject(r" This \ substitution \ is \ invertible \ over \ - \frac{π}{2}<u<\frac{π}{2} \ with \ inverse \ u = \sin^{-1}(x/4).")
  File "/home/mrselukar/miniconda3/lib/python3.8/site-packages/manimlib/mobject/svg/tex_mobject.py", line 147, in __init__
    SingleStringTexMobject.__init__(
  File "/home/mrselukar/miniconda3/lib/python3.8/site-packages/manimlib/mobject

In [13]:
import os
print("Starting to Animate")
retval = os.system('manim solution.py Solution -pl --media_dir "./Animations"')
if retval == 0:
    print("Animation Completed check ./Animations/video for output")
else:
    print("Animation Error Check Manim Logs!!")

Starting to Animate
Animation Error Check Manim Logs!!


In [14]:
"""
# Script to find and replace spaces with " / " in files 
target_names = [item for item in sorted(glob("./Training_Example/latex/*"))]
retval = ""
for path_to_file in target_names:
    print(path_to_file)
    temp = input()
    if temp == "yes":

        file = open(path_to_file)
        retval = file.readlines()
        file.close()
        for i in range(len(retval)):
            print(retval[i])
            x = input()
            if x != '0':
                retval[i] = retval[i].replace(" ", " \ ")
        retval = [x.split("/n")[0][:-1] for x in retval]
    #print(retval)

"""

'\n# Script to find and replace spaces with " / " in files \ntarget_names = [item for item in sorted(glob("./Training_Example/latex/*"))]\nretval = ""\nfor path_to_file in target_names:\n    print(path_to_file)\n    temp = input()\n    if temp == "yes":\n\n        file = open(path_to_file)\n        retval = file.readlines()\n        file.close()\n        for i in range(len(retval)):\n            print(retval[i])\n            x = input()\n            if x != \'0\':\n                retval[i] = retval[i].replace(" ", " \\ ")\n        retval = [x.split("/n")[0][:-1] for x in retval]\n    #print(retval)\n\n'

In [15]:
"""for l in retval:
    print (l)"""

'for l in retval:\n    print (l)'

In [16]:
def func(x_in):
		f = lambda x : eval(x_in)
		return f

In [17]:
a = func('x**2')

In [18]:
[a(i) for i in range(5)]

[0, 1, 4, 9, 16]